In [ ]:
%load_ext autoreload
%autoreload 2
%run __init__
rs1_path = path['RSRCH'](0)

cache_path = './cache/e000'
try:
    os.mkdir(cache_path)
except:
    pass

fig_path = './figures/e000'
try:
    os.mkdir(fig_path)
except:
    pass

import importlib
e000_stim = importlib.import_module('e000-ANK-Stim_Blank')

stdout_orig = sys.stdout

flatui = ["#46637f", "#2ecc71", "#e74c3c", "#3498db", "#9b59b6", "#95a5a6"]
sns.palplot(sns.color_palette("tab10"))
sns.set_palette("tab10")

In [ ]:
NP_CODE = 'NP26'
sel_npref = path['CORE']['RNS']['NP_Ref'][path['CORE']['RNS']['NP_Ref']['NP_code'] == NP_CODE]
sel_ctlg =  path['CORE']['RNS']['CATALOG'][path['CORE']['RNS']['CATALOG']['NP_code'] == NP_CODE]
df_blank = e000_stim.load_stim_detect(NP_CODE)

# Raster of Clip Recordings

In [ ]:
date_implant = pd.Timestamp(sel_npref['Date_First_Implant'].iloc[0])
date_clip = (pd.to_datetime(sel_ctlg['Timestamp']) - date_implant).reset_index(drop=True)
days_clip = (np.array(date_clip) / 1e9 / (24*3600)).astype(float)

ix_longep = np.flatnonzero(sel_ctlg['ECoG trigger'] == 'Long_Episode')
ix_sched = np.flatnonzero(sel_ctlg['ECoG trigger'] == 'Scheduled')

### PLOT Time Distribution of Scheduled Clips
plt.figure(figsize=(12,2), dpi=300)
ax = plt.subplot(111)
ax.vlines(days_clip[ix_sched], 0, 1.0,
          color=(0.0, 0.0, 0.0), linewidth=0.5)
ax.vlines(days_clip[ix_longep], -1.0, 0,
          color=(1.0, 0.78, 0.23), linewidth=0.5)

ax.set_ylim([-2, 2])
ax.spines['left'].set_visible(False)
ax.set_yticks([])
ax.set_xlabel('Days Post-Implant');
ax.set_title('{}: Clip Raster'.format(NP_CODE))
plt.tight_layout()
plt.savefig('{}/{}.{}.svg'.format(fig_path, sel_npref.Initials.iloc[0], 'Clip_Raster'), dpi=300)
plt.show()

# Example Recordings with Stimulation

In [ ]:
for trig, ix in [('Long_Episode', 70),
                ('Scheduled', 1201)]:

    sel = sel_ctlg[sel_ctlg['ECoG trigger'] == trig].reset_index()

    # Load raw data
    df = utils.neuropaceIO.get_ieeg_data_dict(sel.iloc[[ix]], path['CORE']['RNS']['BASE'])
    sel_blank = df_blank[df_blank['Filename'] == sel.iloc[ix]['Filename']]

    spacing = (np.arange(4)*8)

    # Plot the signal
    %matplotlib inline
    plt.figure(figsize=(16,4), dpi=300)
    ax = plt.subplot(111)
    ax.plot(df['sample']['timestamp']-df['sample']['timestamp'][0],
            sp_stats.zscore(df['signal'][:, :], axis=0) + spacing,
            linewidth=0.25, color='k')

    onset = df['sample']['timestamp'][sel_blank['StimOnset_ix']]-df['sample']['timestamp'][0]
    offset = df['sample']['timestamp'][sel_blank['StimOffset_ix']]-df['sample']['timestamp'][0]
    for on_ix, off_ix in zip(onset, offset):
        ax.fill_betweenx([ax.get_ylim()[0], ax.get_ylim()[1]],
                         on_ix, off_ix,
                         color='r', alpha=0.25)

    ax.set_xlabel('Clip Time (sec)')
    ax.set_xticks([0, 30, 60, 90])
    ax.set_yticks(spacing)
    ax.set_yticklabels([sel.iloc[ix]['Ch {} name'.format(ch)] for ch in range(1,5)])

    ax.set_ylim([spacing.min()-8, spacing.max()+8])

    ax.set_title('{}: {}'.format(
        sel.iloc[ix]['NP_code'], sel.iloc[ix]['ECoG trigger']))
    plt.tight_layout()
    plt.savefig('{}/{}.{}.svg'.format(fig_path, sel_npref.Initials.iloc[0], sel.iloc[[ix]].Filename.iloc[0]), dpi=300)
    plt.show()